In [ ]:
!rm -rf ~/.cache/huggingface/datasets
!rm -rf /root/.cache/huggingface/datasets
!rm -rf /content/hf_cache
!rm -rf /root/.cache/huggingface/modules/datasets_modules/datasets/social_bias_frames


In [8]:
import os
from huggingface_hub import login
from huggingface_hub import whoami
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline
from transformers import TrainingArguments
from transformers import Trainer
from transformers import EarlyStoppingCallback
import matplotlib.pyplot as plt
import seaborn as sns
import wandb
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from datasets import Dataset
import evaluate
import torch



In [ ]:


# Authenticate with Hugging Face
login(token=hf_token)
# verifing login
user_info = whoami()
print(f" logged in as: {user_info['name']}")

 logged in as: ms-mielnic


In [9]:
import importlib
import finetuning_eval_func
importlib.reload(finetuning_eval_func)
from finetuning_eval_func import create_eval_summary_df,compute_class_weights, process_csv_social_bias,tokenize_function,custom_collate_fn,evaluation_report,conf_matrix

import re

/Users/mariamielniczuk/anaconda3/envs/capstone_env/bin/python


In [8]:
pd.set_option('display.max_rows', None)

### How to MOUNT THE DRIVE if running in Colab

In [ ]:
# from google.colab import drive
# import os

# path = "/content/drive/MyDrive/capstone_2025/reddit_bias_models/"
# drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Preprocessing the social bias dataset from HuggingFace

In [4]:
import importlib
import finetuning_eval_func
importlib.reload(finetuning_eval_func)
from finetuning_eval_func import create_eval_summary_df,compute_class_weights, process_csv_social_bias,tokenize_function,custom_collate_fn,evaluation_report,conf_matrix

import re

/Users/mariamielniczuk/anaconda3/envs/capstone_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/mariamielniczuk/anaconda3/envs/capstone_env/bin/python
/Users/mariamielniczuk/anaconda3/envs/capstone_env/bin/python


In [7]:
train_df, val_df, test_df, label2id, id2label = process_csv_social_bias('/Users/mariamielniczuk/Documents/capstone')#dir_path is the path to the directory containing the downloaded social bias frames dataset

Train size: 29790
Validation size: 3724
Test size: 3724


/Users/mariamielniczuk/Documents/capstone/finetuning_eval_func.py:76: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['bias_type'].fillna('Neutral', inplace=True)


## Best Hyperparameters search for HateBert using Cross Entropy loss in a bayesian sweep run with WandB

Uncomment if you would like to run the sweep. The uncomented line will upload the results of the sweep

In [10]:
# model_name = "GroNLP/hateBERT"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=len(label2id),
#     id2label=id2label,
#     label2id=label2id,
# )

In [ ]:

# # Compute class weights (inverse frequency)
# from sklearn.utils.class_weight import compute_class_weight

# class_weights = compute_class_weight(
#     class_weight="balanced",
#     classes=np.array(list(label2id.values())),
#     y=train_df['label'].values
# )
# class_weights = torch.tensor(class_weights, dtype=torch.float)

# # Custom Trainer with weighted loss
# class WeightedTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False,num_items_in_batch=None):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.logits
#         loss_fct = CrossEntropyLoss(weight=class_weights.to(model.device))
#         loss = loss_fct(logits, labels)
#         return (loss, outputs) if return_outputs else loss


In [15]:

# train_dataset = Dataset.from_pandas(train_df).map(tokenize_function, batched=True)
# val_dataset   = Dataset.from_pandas(val_df).map(tokenize_function, batched=True)
# test_dataset  = Dataset.from_pandas(test_df).map(tokenize_function, batched=True)

# # Remove extra columns
# for dset in [train_dataset, val_dataset, test_dataset]:
#     dset = dset.remove_columns([col for col in dset.column_names if col not in ["input_ids", "attention_mask", "label"]])


Map: 100%|██████████| 3724/3724 [00:00<00:00, 10499.60 examples/s]


In [ ]:

# accuracy = evaluate.load("accuracy")

# def compute_metrics(p):
#     preds = np.argmax(p.predictions, axis=1)
#     result = accuracy.compute(predictions=preds, references=p.label_ids)

#     # Optionally add more metrics here
#     return result

### Finding the right hyperpareameters with WandB and Bayesian search

In [ ]:
# # Subset to 30% for faster sweeping
# small_train_dataset = train_dataset.select(range(int(0.3 * len(train_dataset))))
# small_val_dataset = val_dataset.select(range(int(0.3 * len(val_dataset))))

In [ ]:
# sweep_config = {
#     "method": "bayes",
#     "metric": {
#         "name": "eval_loss",
#         "goal": "minimize"
#     },
#     "parameters": {
#         "learning_rate": {
#             "min": 1e-6,
#             "max": 5e-5
#         },
#         "weight_decay": {
#             "values": [0.0, 0.01, 0.1]
#         },
#         "per_device_train_batch_size": {
#             "values": [8, 16]
#         },
#         "num_train_epochs": {
#             "values": [1, 2]
#         }
#     }
# }

In [ ]:
# import wandb
# from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# def sweep_train():
#     with wandb.init() as run:
#         config = run.config

#         training_args = TrainingArguments(
#             output_dir=f"./model_{run.id}",
#             do_train=True,
#             do_eval=True,
#             logging_dir="./logs",
#             logging_steps=100,
#             eval_steps=100,
#             save_steps=100,
#             save_total_limit=1,
#             load_best_model_at_end=True,
#             metric_for_best_model="eval_loss",
#             greater_is_better=False,
#             learning_rate=config.learning_rate,
#             weight_decay=config.weight_decay,
#             per_device_train_batch_size=config.per_device_train_batch_size,
#             per_device_eval_batch_size=config.per_device_train_batch_size,
#             num_train_epochs=config.num_train_epochs,
#             run_name=f"sweep_{run.id}",
#             report_to="wandb",
#             eval_strategy="steps" # Changed evaluation_strategy to eval_strategy to match the previous API
#         )

#         trainer = WeightedTrainer(
#             model=model,
#             args=training_args,
#             train_dataset=small_train_dataset,
#             eval_dataset=small_val_dataset,
#             tokenizer=tokenizer,
#             compute_metrics=compute_metrics,
#             callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
#         )

#         trainer.train()

In [11]:
# sweep_id = wandb.sweep(sweep_config, project="hatebert_sbf")
# wandb.agent(sweep_id, function=sweep_train, count=10)

In [6]:
create_eval_summary_df('/Users/mariamielniczuk/Documents/capstone/hatebert_logs')

Found 10 metric files in /Users/mariamielniczuk/Documents/capstone/hatebert_logs


,run_id,eval_loss,eval_accuracy,epoch,step
2,t2835ru3,3.295736,0.718890,2.0,2236.0
0,1cjkn9sp,3.377440,0.717995,2.0,1118.0
3,j4hhkmkl,2.346404,0.716204,1.0,1118.0
4,ym4rv6rt,3.180809,0.715309,2.0,1118.0
5,51acg13c,2.728724,0.714414,2.0,2236.0
7,zbtdpew1,2.816666,0.705461,2.0,1118.0
1,7kxfs0t9,1.731555,0.704566,2.0,2236.0
8,qb8zs1s4,1.194129,0.658908,2.0,1118.0
6,2wuafe8z,1.045755,0.631155,1.0,1118.0
9,p8ueffpb,1.070177,0.582811,1.0,559.0
